# 流式传输

<img src="./assets/LC_streaming.png" width="400">

流式传输减少了数据生成和用户接收之间的延迟。
在智能体中经常使用两种类型：

## 设置

加载和/或检查所需的环境变量

In [ ]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# 从 .env 文件加载环境变量
load_dotenv()

# 检查并打印结果
doublecheck_env("example.env")

In [ ]:
from langchain.agents import create_agent

In [ ]:
agent = create_agent(
    model="openai:gpt-5",
    system_prompt="You are a full-stack comedian",
)

## 无流式传输 (invoke)

In [ ]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

## values（值）
你已经在我们目前的示例中看到了这种流式传输模式。

In [ ]:
# 流式 = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## messages（消息）
消息逐个令牌流式传输数据——可能的最低延迟。这非常适合聊天机器人等交互式应用程序。

In [ ]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

## 工具也可以流式传输！
流式传输通常意味着在最终结果准备好之前将信息传递给用户。在许多情况下这很有用。`get_stream_writer` 写入器允许你轻松地从你创建的源流式传输 `custom` 数据。

In [ ]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """获取给定城市的天气。"""
    writer = get_stream_writer()
    # 流式传输任意数据
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

In [ ]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["custom"],
):
    print(chunk)

## 自己尝试不同的模式！
修改流式模式和选择以产生不同的结果。

In [ ]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])